[pour la visualisation et la compréhension de notre dataset j'ai met en public ce travail sur kaggle ](https://www.kaggle.com/tikoboss/skin-cancer-siim-melanoma)


[pour la création de tfrecords ](https://www.kaggle.com/cdeotte/how-to-create-tfrecords)


In [ ]:
!pip install -q efficientnet
import efficientnet.tfkeras as efn

In [ ]:
import math, re, os
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
import datetime
import tqdm
import json
from collections import Counter
import gc
gc.enable()
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

# Configurations

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
IMAGE_SIZE = [128,128]
BATCH_SIZE = 16* strategy.num_replicas_in_sync

EPOCHS = 15

# Data paths

In [ ]:
from sklearn.model_selection import train_test_split
GCS_DS_PATH = KaggleDatasets().get_gcs_path('oc-d-512512') # you can list the bucket with "!gsutil ls $GCS_DS_PATH"
CLASSES=[0,1,2,3,4,5,6,7]
IMAGE_SIZE = [512, 512] # at this size, a GPU will run out of memory. Use the TPU
EPOCHS = 50
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

from sklearn.model_selection import train_test_split
GCS_PATH    = KaggleDatasets().get_gcs_path('oc-d-512512')

TRAINING_FILENAMES = np.sort(np.array(tf.io.gfile.glob(GCS_PATH + '/train*.tfrec')))


##### split data-set 80% training 20% validation

In [ ]:
TRAINING_FILENAMES,VALIDATION_FILENAMES = train_test_split(TRAINING_FILENAMES,test_size = 0.20,random_state =42)

# Classes

In [ ]:
CLASSES = [0,1]                                                                                                                                               # 100 - 102

# Dataset Functions
baser sur ce   [kernel][1]

[1]: https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "target": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['target'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "filename": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['filename']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled = True, ordered = False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # Diregarding data order. Order does not matter since we will be shuffling the data anyway
    
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
        
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # use data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls = AUTO) # returns a dataset of (image, label) pairs if labeled = True or (image, id) pair if labeld = False
    return dataset

def get_training_dataset():
    
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.repeat() # Since we use custom training loop, we don't need to use repeat() here.
    dataset = dataset.shuffle(20000)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=False)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    
    return dataset  

def get_validation_dataset():
    
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=True)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = int(count_data_items(TRAINING_FILENAMES))
NUM_VALIDATION_IMAGES = int(count_data_items(VALIDATION_FILENAMES))
#NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

print('Dataset: {} training images, {} validation images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES))

# Oversample


In [ ]:
# Get labels and their countings

def get_training_dataset_raw():

    dataset = load_dataset(TRAINING_FILENAMES, labeled=True, ordered=False)
    return dataset


raw_training_dataset = get_training_dataset_raw()

label_counter = Counter()
for images, labels in raw_training_dataset:
    label_counter.update([labels.numpy()])

del raw_training_dataset    
    
label_counting_sorted = label_counter.most_common()

NUM_TRAINING_IMAGES = sum([x[1] for x in label_counting_sorted])
print("number of examples in the original training dataset: {}".format(NUM_TRAINING_IMAGES))

print("labels in the original training dataset, sorted by occurrence")
label_counting_sorted

## nombre de repetition de chaque classe
preparer les fonction qui nous aide  pour faire(le oversampling)
trouver le nombre de repetitions de chaque classe dans le dataset afin d'équilibrer les classes 

In [ ]:
#  Nous voulons que chaque classe se produise au moins (environ) fois `TARGET_MIN_COUNTING`
TARGET_MIN_COUNTING = 100 # chaque classe va se produire 100 fois 

def get_num_of_repetition_for_class(class_id):
    
    counting = label_counter[class_id]
    if counting >= TARGET_MIN_COUNTING:
        return 1.0
    
    num_to_repeat = TARGET_MIN_COUNTING / counting
    
    return num_to_repeat

numbers_of_repetition_for_classes = {class_id: get_num_of_repetition_for_class(class_id) for class_id in range(8)}

print("number of repetitions for each class (if > 1)")
{k: v for k, v in sorted(numbers_of_repetition_for_classes.items(), key=lambda item: item[1], reverse=True) if v > 1}

keys_tensor = tf.constant([k for k in numbers_of_repetition_for_classes])
vals_tensor = tf.constant([numbers_of_repetition_for_classes[k] for k in numbers_of_repetition_for_classes])
table = tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(keys_tensor, vals_tensor), -1)

def get_num_of_repetition_for_example(training_example):
    
    _, label = training_example
    
    num_to_repeat = table.lookup(label)
    num_to_repeat_integral = tf.cast(int(num_to_repeat), tf.float32)
    residue = num_to_repeat - num_to_repeat_integral
    
    num_to_repeat = num_to_repeat_integral + tf.cast(tf.random.uniform(shape=()) <= residue, tf.float32)
    
    return tf.cast(num_to_repeat, tf.int64)

pour le oversampling il ya des techniques specifiques a cause q'uil sagit des fichiers tfrecords donc on peut pas utiliser les  techniques classiques pour le oversampling
telle que smote ....

donc on peut utiliser les poids de cette facon :

    from sklearn.utils import class_weight
    train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
    class_weights = class_weight.compute_class_weight('balanced',
                                                     np.unique(train.target),
                                                     train.target)
    class_weights = dict(enumerate(class_weights))
    
remarque : dans notre class_weights n'ajoute pas vraiment ce que nous attends d'elle  donc je l'ai pas utiliser

aussi on peut utiliser ce code pour le oversampling 

    import pandas as pd
    from collections import Counter

    def get_class_weights(y):
        counter = Counter(y)
        majority = max(counter.values())
        return  {cls: round(float(majority)/float(count), 2) for cls, count in counter.items()}

    train = pd.read('train.csv')
    class_weights = get_class_weights(train.is_attributed.values)
    print(class_weights)

## augmentation 

pour l'augmentation il ya plusieurs techniques a utiliser donc moi jai choiser cette selon cette explication dans ce kernel 

[l'augmentation est baser sur ce kernel](https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96)


pour mes prochain travaux on peut utiliser cette technique :

[source:](https://www.kaggle.com/atamazian/fc-ensemble-external-data-effnet-densenet)

    def random_blockout(img, sl=0.1, sh=0.2, rl=0.4):
        p=random.random()
        if p>=0.25:
            w, h, c = IMAGE_SIZE[0], IMAGE_SIZE[1], 3
            origin_area = tf.cast(h*w, tf.float32)

        e_size_l = tf.cast(tf.round(tf.sqrt(origin_area * sl * rl)), tf.int32)
        e_size_h = tf.cast(tf.round(tf.sqrt(origin_area * sh / rl)), tf.int32)

        e_height_h = tf.minimum(e_size_h, h)
        e_width_h = tf.minimum(e_size_h, w)

        erase_height = tf.random.uniform(shape=[], minval=e_size_l, maxval=e_height_h, dtype=tf.int32)
        erase_width = tf.random.uniform(shape=[], minval=e_size_l, maxval=e_width_h, dtype=tf.int32)

        erase_area = tf.zeros(shape=[erase_height, erase_width, c])
        erase_area = tf.cast(erase_area, tf.uint8)

        pad_h = h - erase_height
        pad_top = tf.random.uniform(shape=[], minval=0, maxval=pad_h, dtype=tf.int32)
        pad_bottom = pad_h - pad_top

        pad_w = w - erase_width
        pad_left = tf.random.uniform(shape=[], minval=0, maxval=pad_w, dtype=tf.int32)
        pad_right = pad_w - pad_left

        erase_mask = tf.pad([erase_area], [[0,0],[pad_top, pad_bottom], [pad_left, pad_right], [0,0]], constant_values=1)
        erase_mask = tf.squeeze(erase_mask, axis=0)
        erased_img = tf.multiply(tf.cast(img,tf.float32), tf.cast(erase_mask, tf.float32))

        return tf.cast(erased_img, img.dtype)
    else:
        return tf.cast(img, img.dtype)

In [ ]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))


def transform(image, label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 #fix for size 331
    
    rot = 15. * tf.random.normal([1],dtype='float32')
    shr = 5. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    h_shift = 16. * tf.random.normal([1],dtype='float32') 
    w_shift = 16. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3]), label

[pour voir le test sur l'augmentation j'ai créer un kernel sur kaggle ou je montre le fonctionement de notre augmentation avec les images ](https://www.kaggle.com/tikoboss/test-of-aug-by-tko-okt?scriptVersionId=42269313)

## oversampled training dataset

In [ ]:
def get_training_dataset_with_oversample(repeat_dataset=True, oversample=False, augumentation=False):

    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)

    if oversample:
        dataset = dataset.flat_map(lambda image, label: tf.data.Dataset.from_tensors((image, label)).repeat(get_num_of_repetition_for_example((image, label))))

    if augumentation:
        dataset = dataset.map(transform, num_parallel_calls=AUTO)
    
    if repeat_dataset:
        dataset = dataset.repeat() # the training dataset must repeat for several epochs
    
    dataset = dataset.shuffle(20000)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    
    return dataset

## test for oversampled dataset

In [ ]:
oversampled_training_dataset = get_training_dataset_with_oversample(repeat_dataset=False, oversample=True, augumentation=False)

label_counter_2 = Counter()
for images, labels in oversampled_training_dataset:
    label_counter_2.update(labels.numpy())

del oversampled_training_dataset

label_counting_sorted_2 = label_counter_2.most_common()

NUM_TRAINING_IMAGES_OVERSAMPLED = sum([x[1] for x in label_counting_sorted_2])
print("number of examples in the oversampled training dataset: {}".format(NUM_TRAINING_IMAGES_OVERSAMPLED))

print("labels in the oversampled training dataset, sorted by occurrence")
label_counting_sorted_2

dans d'autres problemes on peut utiliser cette fonction f1_loss pour minimiser la perte de f1_score ,qui fait que f1_score va augmenter

    def f1_loss(y_true, y_pred):

        tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
        tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
        fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
        fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

        p = tp / (tp + fp + K.epsilon())
        r = tp / (tp + fn + K.epsilon())

        f1 = 2*p*r / (p+r+K.epsilon())
        f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
        return 1 - K.mean(f1)

pour calculer f1_score avec macro aussi je suggéré cette façon d'utilisation de f1_score
metrics = [tfa.metrics.f_scores.F1Score(num_classes=2,average="macro")] 


    def fbeta_score_macro(y_true, y_pred, beta=1, threshold=0.1):

        y_true = K.cast(y_true, 'float')
        y_pred = K.cast(K.greater(K.cast(y_pred, 'float'), threshold), 'float')

        tp = K.sum(y_true * y_pred, axis=0)
        fp = K.sum((1 - y_true) * y_pred, axis=0)
        fn = K.sum(y_true * (1 - y_pred), axis=0)

        p = tp / (tp + fp + K.epsilon())
        r = tp / (tp + fn + K.epsilon())

        f1 = (1 + beta ** 2) * p * r / ((beta ** 2) * p + r + K.epsilon())
        f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
        return K.mean(f1)

In [ ]:
"""
import tensorflow as tf
from keras.losses import binary_crossentropy, categorical_crossentropy
import keras.backend as K
import numpy as np
from prettytable import PrettyTable
from prettytable import ALL
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)


def fbeta_score_macro(y_true, y_pred, beta=1, threshold=0.1):

    y_true = K.cast(y_true, 'float')
    y_pred = K.cast(K.greater(K.cast(y_pred, 'float'), threshold), 'float')

    tp = K.sum(y_true * y_pred, axis=0)
    fp = K.sum((1 - y_true) * y_pred, axis=0)
    fn = K.sum(y_true * (1 - y_pred), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = (1 + beta ** 2) * p * r / ((beta ** 2) * p + r + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)
    
"""

# learning rate

In [ ]:
# Learning rate schedule for TPU, GPU and CPU.
# Using an LR ramp up because fine-tuning a pre-trained model.
# Starting with a high LR would break the pre-trained weights.

# pour fine-tuning https://flyyufelix.github.io/2016/10/03/fine-tuning-in-keras-part1.html

LR_START = 0.00001
LR_MAX = 0.00005 * strategy.num_replicas_in_sync
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

rng = [i for i in range(25 if EPOCHS<25 else EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

> #aussi pour learning rate 

*  source [get_cosine_schedule_with_warmup](https://huggingface.co/transformers/_modules/transformers/optimization.html#get_cosine_schedule_with_warmup)

aussi pour LR on peut utiliser cette fonction que je trouve vraiment efficace 
[source ](https://www.kaggle.com/chankhavu/a-beginner-s-tpu-kernel-single-model-0-97)

In [ ]:
import kernel_tensorflow_utils as ktu
lr_callback = ktu.LRSchedulers.FineTuningLR(
    
    lr_start=1e-5, lr_max=5e-5 * strategy.num_replicas_in_sync, lr_min=1e-5,
    lr_rampup_epochs=5, lr_sustain_epochs=0, lr_exp_decay=0.8, verbose=1)

plt.figure(figsize=(8, 5))
lr_callback.visualize(steps_per_epoch=NUM_TRAINING_IMAGES, epochs=40)

In [ ]:
valid_ds = get_validation_dataset()

valid_images_ds = valid_ds.map(lambda image, label: image)
valid_labels_ds = valid_ds.map(lambda image, label: label).unbatch()

valid_labels = next(iter(valid_labels_ds.batch(NUM_VALIDATION_IMAGES))).numpy() # get everything as one batch

valid_steps = NUM_VALIDATION_IMAGES // BATCH_SIZE

if NUM_VALIDATION_IMAGES % BATCH_SIZE > 0:
    valid_steps += 1

# Train +original dataset
en utilisant dataset tele quelle est ,pas de aug et pas de oversample

### DenseNet201 sans oversample et sans augumentation
j'ai déjà essayé **densnet121** et **densnet169** donc les meilleurs résultats c'est en utilisant DenseNet201
,aussi **densenet201** sans oversample et avec aug il n'ya pas vraiment de changement en terme de résultats par contre une perte de 2% en niveau de f1_score et une grande perte de 4% en niveau de accuracy

**remarque** les autres architectures telle que **vgg16** et **vgg19** et **rasnet50**..... ne donne pas de bonne résultats 
par contre j'ai pas essayer **rasnet34** et **rasnet18** ( a base de pytorsh ) 
**rasnext50** j'ai vu un kernel sur ca en utilisant l'ensemble learning mais pas pour le probleme de **skin cancer** donc j'ai pas vraiment essayer ca

pour lensemble learning (ensemble de efficienet de b0->b7)  j'ai essayer cette facont mais j'ai eu les memes résultats q'un seul model en + le probleme  cest lors de l'exécution ça prend 3h donc c'est pas le bon choix  

par contre j'ai pas essayer l'ensemble learning de (densenet201,densenet121,densenet169)

### aussi je suggère cette technique dans les prochain travaux
#### BiLinear EfficientNet Focal Loss+ Label Smoothing
#### je pense il y'a un seul un articele qui a utiliser cette technique
[source](https://www.kaggle.com/jimitshah777/bilinear-efficientnet-focal-loss-label-smoothing)

pour la perte j'ai essayer cette fonction qui abouti vraiment a de bonne resultas en terme de auc 

    def focal_loss(alpha=0.25,gamma=2.0):
        def focal_crossentropy(y_true, y_pred):
            bce = K.binary_crossentropy(y_true, y_pred)

            y_pred = K.clip(y_pred, K.epsilon(), 1.- K.epsilon())
            p_t = (y_true*y_pred) + ((1-y_true)*(1-y_pred))

            alpha_factor = 1
            modulating_factor = 1

            alpha_factor = y_true*alpha + ((1-alpha)*(1-y_true))
            modulating_factor = K.pow((1-p_t), gamma)

            # compute the final loss and return
            return K.mean(alpha_factor*modulating_factor*bce, axis=-1)
        return focal_crossentropy
        
[source focal_loss](https://www.kaggle.com/c/santander-customer-transaction-prediction/discussion/83363)

In [ ]:
!pip install livelossplot
from livelossplot import PlotLossesKeras
cb=[PlotLossesKeras()]

In [ ]:
def focal_loss(alpha=0.25,gamma=2.0):
    def focal_crossentropy(y_true, y_pred):
        bce = K.binary_crossentropy(y_true, y_pred)

        y_pred = K.clip(y_pred, K.epsilon(), 1.- K.epsilon())
        p_t = (y_true*y_pred) + ((1-y_true)*(1-y_pred))

        alpha_factor = 1
        modulating_factor = 1

        alpha_factor = y_true*alpha + ((1-alpha)*(1-y_true))
        modulating_factor = K.pow((1-p_t), gamma)

        # compute the final loss and return
        return K.mean(alpha_factor*modulating_factor*bce, axis=-1)
    return focal_crossentropy

In [ ]:
original_training_dataset = get_training_dataset_with_oversample(repeat_dataset=True, oversample=False, augumentation=False)

import keras

with strategy.scope():
    base_network = efn.EfficientNetB7(input_shape=(512,512,3),weights='noisy-student',include_top=False)
    network = keras.Sequential()
    network.add(base_network)
    network.add(keras.layers.MaxPooling2D())
    network.add(keras.layers.Conv2D(2810,3,padding='same'))
    network.add(keras.layers.Dropout(0.25))
    network.add(keras.layers.BatchNormalization())
    network.add(keras.layers.ReLU())
    network.add(keras.layers.GlobalAveragePooling2D())

    network.add(keras.layers.Dense(1024))
    network.add(keras.layers.Dropout(0.25))
    network.add(keras.layers.BatchNormalization())
    network.add(keras.layers.LeakyReLU())

    network.add(keras.layers.Dense(512))
    network.add(keras.layers.Dropout(0.25))
    network.add(keras.layers.BatchNormalization())
    network.add(keras.layers.LeakyReLU())

    network.add(keras.layers.Dense(256))
    network.add(keras.layers.Dropout(0.25))
    network.add(keras.layers.BatchNormalization())
    network.add(keras.layers.LeakyReLU())


    network.add(keras.layers.Dense(8,activation='softmax'))
    network.compile(optimizer=keras.optimizers.Adam(lr=0.00001),loss=keras.losses.SparseCategoricalCrossentropy(),metrics=[keras.metrics.SparseCategoricalAccuracy()])
    network.summary()

In [ ]:
network.summary()
from keras.utils.vis_utils import plot_model
plot_model(network, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
history = network.fit(
    original_training_dataset, 
    steps_per_epoch=NUM_TRAINING_IMAGES // BATCH_SIZE,
    epochs=12,
    callbacks=[lr_callback,cb],
    validation_data=valid_ds
)

valid_probs =network.predict(valid_images_ds, steps=valid_steps)
valid_preds = np.argmax(valid_probs, axis=-1)

del network
gc.collect()
tf.keras.backend.clear_session()

val_acc = history.history['val_sparse_categorical_accuracy']

score = f1_score(valid_labels, valid_preds, labels=range(len(CLASSES)), average='macro')
acc = accuracy_score(valid_labels, valid_preds)
precision = precision_score(valid_labels, valid_preds, labels=range(len(CLASSES)), average='macro')
recall = recall_score(valid_labels, valid_preds, labels=range(len(CLASSES)), average='macro')

print("results")
print("best 10 validation accuracies = {}".format(sorted(val_acc, reverse=True)[:10]))
print('f1 score: {:.6f} | recall: {:.6f} | precision: {:.6f} | acc: {:.6f}'.format(score, recall, precision, acc))

In [ ]:
import sklearn.metrics
print(sklearn.metrics.classification_report(valid_labels, valid_preds))

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(valid_labels, valid_preds, labels=[0,1,2,3,4,5,6,7])
cm